In [3]:
# Import libraries
seed_value= 0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.random.set_seed(0)
import gradio as gr
import time
from Image_Segmentation.model import *
from Image_Segmentation.data import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import numpy as np
from keras.preprocessing import image
import time
import matplotlib.pyplot as plt
import glob
import os
import cv2
import PIL
from keras.preprocessing.image import img_to_array,load_img
from keras.applications.vgg16 import preprocess_input,decode_predictions
from keras.models import load_model
from fastai.vision import *
import matplotlib.image as mpimg
# Load learners and models
learn = load_learner('Crown_Rump_Identifier/densenet121_hp_2')
model_classification = load_model('Segmentation_Classifier/model/VGG16_model3.hdf5')
model = load_model("Image_Segmentation/model/UNetModel.hdf5", custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
# Data Pre-Processing function called for the U-Net model prediction
dp = data_preprocess(test_path='Image_Segmentation/data/test',save_path='temp',
                     test_fullscale_path='Image_Segmentation/data/test_empty',
                     save_post_path='Image_Segmentation/data/test_result/post/post', save_roi_restored='data/roi/restored', 
                     target_rows = 256, target_cols = 256, img_type = 'jpg')
# Find Crown and Rump Predictions
def findCRPreds(model, image) -> None:
    # Call prediction model
    prediction = model.predict(image)
    cr_locations = prediction[1]
    coord = cr_locations[0]
    coord1 = cr_locations[1]
    # Scale the coordinates
    r1 = (float(coord[0])+1)/2
    c1 = (float(coord[1])+1)/2
    r2 = (float(coord1[0])+1)/2
    c2 = (float(coord1[1])+1)/2
    r1_r = (r1 * image.size[0])
    r2_r = (r2 * image.size[0])
    c1_r = (c1 * image.size[1])
    c2_r = (c2 * image.size[1])
    location = tensor(
      [ [float(c1_r), float(r1_r) ], 
        [float(c2_r), float(r2_r) ]  ]
       )
    # Plot and save the coordinates
    img = PIL.Image.open("temp/segmented_image_after_mask.jpg")
    gray_img = img.convert("L")
    plt.imshow(gray_img, cmap='gray')
    x, y = location.T
    plt.scatter(x,y)
    plt.axis('off')
    plt.savefig('temp/my_plot.jpg',  bbox_inches='tight',pad_inches = 0)
    im = plt.imread('temp/my_plot.jpg')
    implot = plt.imshow(im)
    plt.show()

# Function called during the Gradio interface
def segmentation(image):
    # Load image for segmentation model
    picture = image.save("temp/segmented.jpg") 
    x,img_size = dp.image_normalized("temp/segmented.jpg")
    image_seg = model.predict(x)
    name = "segmented_image2"
    # Save predicted mask
    dp.saveResult(image_seg, img_size, name.split('.')[0])
    # Segment original image with the use of the mask
    MaskImg = cv2.imread('temp/segmented_image2_predict.jpg', 0) 
    image_seg = cv2.imread('temp/segmented.jpg', 0) 
    bitwiseAnd = cv2.bitwise_and(image_seg, MaskImg)
    cv2.imwrite('temp/segmented_image_after_mask.jpg',bitwiseAnd)
    # Load segmented image
    image1 = load_img('temp/segmented_image_after_mask.jpg', target_size=(224, 224)) 
    # Pre-process segmented image
    image_x = img_to_array(image1)
    image_x = np.expand_dims(image_x, axis=0)
    image_x = tf.keras.applications.vgg16.preprocess_input(image_x, data_format=None)
    preds = model_classification.predict(image_x)
    # Classify predictions for classification
    if preds[0][0]>preds[0][1]:
        Result = 'Good Segmentation'
    elif preds[0][1]>preds[0][0]:
        Result = 'Bad Segmentation'
    SegmentedImage2 = open_image('temp/segmented_image_after_mask.jpg')
    # Predict Crown and Rump Coordinates
    findCRPreds(learn, SegmentedImage2)
    CrownRumpImg = load_img('temp/my_plot.jpg') 
    SegmentedImg = load_img('temp/segmented_image_after_mask.jpg')
    # Return outputted images
    return SegmentedImg, Result, CrownRumpImg
# Configuration of outputs
o1 = gr.outputs.Image(type="pil", label="Segmented Image")
o2 = gr.outputs.Textbox()
o3 = gr.outputs.Image(type="pil", label="Crown and Rump Identified")
# Gradio interface
gr.Interface(fn=segmentation, 
              inputs=gr.inputs.Image(type="pil"), 
              outputs=[o1,o2,o3]).launch();

Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\samsu\anaconda3\envs\WebApp_Fatsai\lib\site-packages\gradio\routes.py", line 214, in predict
    output = await run_in_threadpool(app.launchable.process_api, body, username)
  File "C:\Users\samsu\anaconda3\envs\WebApp_Fatsai\lib\site-packages\starlette\concurrency.py", line 39, in run_in_threadpool
    return await anyio.to_thread.run_sync(func, *args)
  File "C:\Users\samsu\anaconda3\envs\WebApp_Fatsai\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "C:\Users\samsu\anaconda3\envs\WebApp_Fatsai\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "C:\Users\samsu\anaconda3\envs\WebApp_Fatsai\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "C:\Users\samsu\anaconda3\envs\WebApp_Fatsai\lib\site-packa